<a href="https://colab.research.google.com/github/BlackSparrow-43/deep-rl-class/blob/main/My_Projects/DQN/CartPole/manual_CartPole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import Sequential, clone_model, load_model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from keras.callbacks import TensorBoard

import gym
import random
import numpy as np 
import time
from collections import deque

In [ ]:
class Dqn():
    
    def __init__(self,Load_model = None):
                
        self.Q_model = self.create_model()
        self.T_model = self.create_model()
        #self.tensorboard = tf.keras.callbacks.TensorBoard(log_dir="logs/{}-{}".format("rl_cartpole", int(time.time())),histogram_freq=1)
        self.epsilon = 1.0
        self.max_epsilon = 1.0
        self.min_epsilon = 0.009
        self.decay_rate = .01
        self.gamma = 0.99
        self.batch_size = 100
        self.replay_mem_size = 50_000
        self.update_T_model = 10
        self.EPOCH = 150
        self.log_interval = 10
        self.replay_mem = deque(maxlen=self.replay_mem_size)
        #self.Load_model = Load_model

  
    def create_model(self,verbose=0):

        model = Sequential()
        
        model.add(Dense(16,  input_shape=(1, env.observation_space.shape[0])))
        model.add(Activation('relu'))

        model.add(Dense(32))
        model.add(Activation('relu'))

        model.add(Dense(env.action_space.n))
        model.add(Activation('linear'))
        
        model.compile(loss = "mse", optimizer = Adam(lr = .001), metrics = ["accuracy"])
        
        if verbose == 1:
            print(model.summary())
        return model
    

    def replay_buffer(self,epoch):
        
        if len(self.replay_mem) < self.batch_size :
            return
        
        minibatch = random.sample(self.replay_mem, self.batch_size)
        
        current_states = np.array([history[0] for history in minibatch])
        cur_state_q_list = self.Q_model.predict(current_states)
        
        next_states = np.array([history[3] for history in minibatch])
        next_state_q_list = self.T_model.predict(next_states)
        
        features = []
        labels = []
        for index,(current_state, action, reward, next_state, done) in enumerate(minibatch):
            
            if not done:
                max_q_value = np.max(next_state)
                new_q = reward + self.gamma*max_q_value
            else:
                new_q = reward
            
            current_q_state = cur_state_q_list[index]
            current_q_state[0][action] = new_q

            features.append(current_state)
            labels.append(current_q_state)
        self.Q_model.fit(np.array(features)/255, np.array(labels), batch_size=self.batch_size, verbose=0, shuffle=False )
        
    

    def update_model_handler(self,epoch):
        if epoch > 0 and epoch % self.update_T_model == 0:
            self.T_model.set_weights(self.Q_model.get_weights())
    
            
    def epsilon_greedy_action_selection(self, epsilon, obs):
        select=None
        if np.random.random() > self.epsilon:
            prediction = self.Q_model.predict(np.array(obs).reshape(-1, *obs.shape)/255)  # perform the prediction on the observation
            action = np.argmax(prediction)  
            select="from_table"# Chose the action with the higher value
        else:
            action = np.random.randint(0, env.action_space.n)  # Else use random action
            select="random"
        
        return action, select
    
    
    def epsilon_reduce_exp(self,epoch):
        return (self.min_epsilon+(self.max_epsilon-self.min_epsilon)*np.exp((-self.decay_rate)*epoch))
    
    
    def reward_system(self, reward, obs, done, step):
    
        angle = obs[0][2]
        pos = obs[0][0]
        reward_step = 0  
        
        if -6 <= angle < -5:
            reward_ang = -60
        elif -5 <= angle < -4.5:
            reward_ang = -48
        elif -4.5 <= angle < -4:
            reward_ang = -36
        elif -4 <= angle < -3.5:
            reward_ang = -24
        elif -3.5 <= angle < -3:
            reward_ang = -16
        elif -3 <= angle < -2.5:
            reward_ang = -8
        elif -2.5 <= angle < -2:
            reward_ang = -4
        elif -2 <= angle < -1.5:
            reward_ang = -2
        elif -1.5 <= angle < -1:
            reward_ang = -1
        elif -1 <= angle < -.5:
            reward_ang = 0
        elif -0.5 <= angle < 0.5:
            reward_ang = 1
        elif 1 <= angle < 1.5 :
            reward_ang = 0
        elif 1.5 <= angle < 2:
            reward_ang = -1
        elif 2 <= angle < 2.5:
            reward_ang = -2
        elif 2.5 <= angle < 3:
            reward_ang = -4
        elif 3 <= angle < 3.5:
            reward_ang = -8
        elif 3.5 <= angle < 4:
            reward_ang = -12
        elif 4 <= angle < 4.5:
            reward_ang = -18
        elif 4.5 <= angle < 5:
            reward_ang = -26
        elif 5 <= angle < 5.5:
            reward_ang = -36
        elif 5.5 <= angle < 6:
            reward_ang = -44
        elif 6 <= angle < 6.5:
            reward_ang = -56
        elif 6.5 <= angle < 7:
            reward_ang = -64
        else:
            reward_ang = -100


        if -3 <= pos < -2 :
          reward_pos = -1
        elif -2 <= pos < -1:
          reward_pos = 0
        elif -1 <= pos < 1:
          reward_pos = 1
        elif 1 <= pos < 2:
          reward_pos = 0
        elif 2 <= pos < 3:
          reward_pos = -1
        else:
          reward_pos = -5

        if done and step < 300:
            reward_step = -300
    
        return reward_ang + reward_pos  + 1
    
    
    
    def training(self):
        
        best_steps = 0
        total_rewards = 0
        sum_rewards = 0
        
        for epoch in range(self.EPOCH):
            obs = env.reset()
            obs = obs.reshape([1,4])
            done = False
            step = 0
            table_nos, random_nos = 0, 0 
            while not done:
                action, select = self.epsilon_greedy_action_selection(self.epsilon, obs)
                next_obs, reward, done, info = env.step(action)
                next_obs = next_obs.reshape([1,4])
                mod_reward = self.reward_system(reward, next_obs, done, step)
                sum_rewards += mod_reward
                self.replay_mem.append((obs, action, mod_reward, next_obs, done))
                obs = next_obs
                step += 1
                self.replay_buffer(epoch)
                
                if select == "from_table":
                    table_nos+=1
                elif select == "random":
                    random_nos+=1
                
            self.epsilon = self.epsilon_reduce_exp(epoch)
            self.update_model_handler(epoch)
            
            if step > best_steps:
                best_steps = step
                
            if epoch % self.log_interval == 0:
                table_per = 100 *(table_nos / (table_nos + random_nos))
                random_per = 100 *(random_nos / (table_nos + random_nos))
                total_rewards += sum_rewards
                print("Gen:"+str(epoch),"random="+str(round(random_per,3))+"%","Neural="+str(round(table_per,3))+"%", 
                      "epsilon="+str(self.epsilon), "rewards="+str(sum_rewards), "Total_rewards="+str(total_rewards),
                      "steps="+str(step), "most_step="+str(best_steps))
                sum_rewards = 0
        print("Gen:"+str(epoch),"random="+str(round(random_per,3))+"%","Neural="+str(round(table_per,3))+"%", "epsilon="+str(self.epsilon), "rewards="+str(sum_rewards), 
            "Total_rewards="+str(total_rewards),"steps="+str(step), "most_step="+str(best_steps))


    def play(self):
        total_reward = 0
        obs = env.reset()
        then = time.time()
        step=0
        while True:
            obs = obs.reshape([1,4])
            prediction = self.Q_model.predict(np.array(obs).reshape(-1, *obs.shape)/255)
            action = np.argmax(prediction)  
            new_obs, reward, done, info = env.step(action)
            total_reward += self.reward_system(reward, obs, done, step)
            obs = new_obs
            step +=1
            if done:
                break
        print("Time Taken="+str(round(then-time.time(),3))+"s")
        print("Rewards="+str(total_reward))
        env.close()

In [ ]:
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

In [ ]:
env_name = 'CartPole-v1'
env = gym.make(env_name)
agent = Dqn()
agent.training()
agent.play()

Gen:0 random=100.0% Neural=0.0% epsilon=1.0 rewards=54 Total_rewards=54 steps=18 most_step=18
Gen:10 random=77.143% Neural=22.857% epsilon=0.9056938812736359 rewards=690 Total_rewards=744 steps=35 most_step=59
Gen:20 random=100.0% Neural=0.0% epsilon=0.82036217630028 rewards=639 Total_rewards=1383 steps=12 most_step=59
Gen:30 random=76.923% Neural=23.077% epsilon=0.7431508566955825 rewards=471 Total_rewards=1854 steps=13 most_step=59
Gen:40 random=68.421% Neural=31.579% epsilon=0.6732871656213186 rewards=570 Total_rewards=2424 steps=19 most_step=59
Gen:50 random=73.684% Neural=26.316% epsilon=0.6100718837752197 rewards=465 Total_rewards=2889 steps=19 most_step=59
Gen:60 random=54.545% Neural=45.455% epsilon=0.5528723313691801 rewards=456 Total_rewards=3345 steps=11 most_step=59
Gen:70 random=64.286% Neural=35.714% epsilon=0.5011160360572867 rewards=426 Total_rewards=3771 steps=14 most_step=59
Gen:80 random=72.727% Neural=27.273% epsilon=0.4542850034401666 rewards=381 Total_rewards=4152

In [ ]:
Q_model.save_weights('./checkpoints/my_checkpoint')

In [ ]:
agent.Q_model.save("/content/drive/MyDrive/Colab Notebooks/dqn_cartpole")

In [ ]:
import keras_tuner
from tensorflow.keras import layers, models, optimizers

class models:
  def build_model(self,hp):
      model = Sequential()
      model.add(tf.keras.Input(shape=(1, env.observation_space.shape[0])))
      model.add(Activation("relu"))
    # Tune the number of layers.
      for i in range(hp.Int("num_layers", 1, 3)):
          model.add(
              layers.Dense(
                  # Tune number of units separately.
                  units=hp.Int(f"units_{i}", min_value=6, max_value=64, step=4),
                  activation="softmax"
              )
          )
      if hp.Boolean("dropout"):
          model.add(layers.Dropout(rate=0.25))
      model.add(
          layers.Dense(
              env.action_space.n, 
              activation=hp.Choice("activation", ["softmax","linear"]),
              )
          )
      learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
      loss_function = hp.Choice("loss", ["categorical_crossentropy","mse"])
      model.compile(
          optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
          loss=loss_function,
          metrics=["accuracy"],
      )
      return model

  def mysearch(self):  
    self.mytuner = keras_tuner.RandomSearch(
      hypermodel=self.build_model,
      objective="accuracy",
      max_trials=5,
      executions_per_trial=4,
      overwrite=True,
      directory="/content/drive/MyDrive/Colab Notebooks/DQN",
      project_name="DQN_cartpole",
    )
    self.mytuner.search_space_summary()
model = models()
model.mysearch()
